In [ ]:
%load_ext autoreload
%autoreload 2

# Unbiased ECMWF

Here we propose a small model which is a debiased ECMWF forecast according to the data we have.
Compute and score the climatology, to use as a baseline.

In [ ]:
import dask
import dask.array as da
import dask.distributed
import dask_jobqueue
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import scipy.stats
import xarray as xr

In [ ]:
from crims2s.util import fix_dataset_dims

In [ ]:
INPUT_TRAIN = '***BASEDIR***training-input/0.3.0/netcdf'
OBSERVATIONS = '***BASEDIR***training-output-reference/'

## Boost dask cluster

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ***HOME***.bash_profile','conda activate s2s'],
)

In [ ]:
cluster.scale(jobs=3)  # Scale to two working nodes as configured.
client = dask.distributed.Client(cluster)

In [ ]:
client

# Open model data

In [ ]:
CENTER = 'ecmwf'
FIELD = 't2m'

In [ ]:
input_path = pathlib.Path(INPUT_TRAIN)

In [ ]:
files = sorted([f for f in input_path.iterdir() if CENTER in f.stem and FIELD in f.stem])

In [ ]:
files[:10]

In [ ]:
one_ecmwf = xr.open_dataset(files[0])

In [ ]:
one_ecmwf

In [ ]:
fix_dataset_dims(one_ecmwf)

In [ ]:
ecmwf = xr.open_mfdataset(files, preprocess=fix_dataset_dims)

In [ ]:
ecmwf

In [ ]:
ecmwf_train = ecmwf.sel(forecast_year=slice(None, 2018))
ecmwf_val = ecmwf.sel(forecast_year=slice(2019, None))

In [ ]:
ecmwf_train

In [ ]:
ecmwf_val

In [ ]:
ecmwf_mean = ecmwf.mean(dim='realization')
ecmwf_std = ecmwf.std(dim='realization')

In [ ]:
ecmwf_mean

## Open observations

In [ ]:
obs_path = pathlib.Path(OBSERVATIONS)
obs_files = [f for f in obs_path.iterdir() if 't2m' in f.stem]

In [ ]:
obs = xr.open_mfdataset(obs_files, preprocess=fix_dataset_dims).isel(lead_time=slice(1, None))

In [ ]:
obs

In [ ]:
obs_train = obs.sel(forecast_year=slice(None, 2018))
obs_val = obs.sel(forecast_year=slice(2019, None))

In [ ]:
obs_train

In [ ]:
climatology_mean = obs_weeks34.mean(dim=['forecast_year', 'lead_time']).compute()

In [ ]:
climatology_std = obs_weeks34.std(dim=['forecast_year', 'lead_time']).compute()

In [ ]:
climatology_mean.isel(forecast_dayofyear=0).t2m.plot()

In [ ]:
climatology_std.isel(forecast_dayofyear=0).t2m.plot()

## Compute model bias

In [ ]:
ecmwf_train_mean = ecmwf_train.mean(dim='realization')

In [ ]:
ecmwf_train_mean

In [ ]:
ecmwf_train_bias = ecmwf_train_mean - obs_train

In [ ]:
ecmwf_train_bias_week34 = ecmwf_train_bias.sel(lead_time=slice(datetime.timedelta(days=14), datetime.timedelta(days=28))).mean(dim=['forecast_year', 'lead_time'])

In [ ]:
ecmwf_train_bias_week34

In [ ]:
ecmwf_train_bias_week34.isel(forecast_dayofyear=10).t2m.plot()

In [ ]:
ecmwf_val_w34 = ecmwf_val.sel(lead_time=slice('14D', '27D'))

In [ ]:
ecmwf_val_w34

In [ ]:
ecmwf_val_w34_debiased = ecmwf_val_w34 - ecmwf_train_bias_week34

In [ ]:
ecmwf_val_w34_debiased_mean = ecmwf_val_w34_debiased.mean(dim=['realization', 'lead_time'])

In [ ]:
ecmwf_val_w34_debiased_mean

In [ ]:
ecmwf_val_w34_debiased_std = ecmwf_val_w34_debiased.std(dim=['realization', 'lead_time'])

In [ ]:
ecmwf_val_w34_debiased_std

In [ ]:
obs_train

In [ ]:
obs_val - ecmwf_val_w34_debiased_mean

In [ ]:
obs_val_w34 = obs_val.sel(lead_time=slice('14D', '27D'))

In [ ]:
obs_val_w34

In [ ]:
obs_train_w34 = obs_train.sel(lead_time=slice('14D', '27D'))

In [ ]:
obs_train_w34

In [ ]:
thresholds = obs_train_w34.quantile([.33, .66], dim=['forecast_year', 'lead_time'])

In [ ]:
thresholds

In [ ]:
thresholds.isel(quantile=1, forecast_dayofyear=0).t2m.plot()

In [ ]:
ecmwf_val_w34_debiased_mean
ecmwf_val_w34_debiased_std

In [ ]:
forecast_distribution = scipy.stats.norm(loc=ecmwf_val_w34_debiased_mean.t2m.data, scale=ecmwf_val_w34_debiased_std.t2m.data)

In [ ]:
p_below = forecast_distribution.cdf(thresholds.isel(quantile=0).t2m.data)

In [ ]:
p_below = _

In [ ]:
p_below.isnan()

In [ ]:
np.isnan(p_below).sum()

In [ ]:
p_below.shape

In [ ]:
fig, ax = plt.subplots()
cax = ax.imshow(p_below[0,30])
fig.colorbar(cax, ax=ax)

In [ ]:
p_above = forecast_distribution.cdf(thresholds.isel(quantile=1).t2m.data)

In [ ]:
p_normal = p_above - p_below

In [ ]:
p_normal.shape

In [ ]:
p_above = np.ones_like(p_above) - p_above

In [ ]:
(p_above + p_normal + p_below)[0, 0][~np.isnan(p_above)[0, 0]].min()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(p_normal[0, 0])

In [ ]:
forecast = np.stack([p_below, p_normal, p_above])

In [ ]:
forecast.shape

In [ ]:
forecast = xr.DataArray(forecast, dims=['category', 'forecast_year', 'forecast_dayofyear', 'latitude', 'longitude'], coords={
    'category': xr.DataArray(['below normal', 'within normal', 'above normal'], dims='category'),
})

In [ ]:
forecast = forecast.assign_coords({
    'forecast_dayofyear': obs_train_w34.forecast_dayofyear,
    'latitude': obs_train_w34.latitude,
    'longitude': obs_train_w34.longitude,
    'forecast_time': obs_val_w34.forecast_time
})

In [ ]:
dataset = xr.Dataset({'t2m': forecast})

In [ ]:
dataset

In [ ]:
dataset.t2m.isel(category=2,  forecast_dayofyear=0).plot()